In [1]:
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

In [2]:
train_df = pd.read_csv('./dataset/final/train.csv', usecols=['source', 'label'])
train_df['source'] = train_df['source'].str.lower()
train_df['label'] = train_df['label'].str.lower()
train_ds = Dataset.from_pandas(train_df)

test_df = pd.read_csv('./dataset/final/test.csv', usecols=['source', 'label'])
test_df['source'] = test_df['source'].str.lower()
test_df['label'] = test_df['label'].str.lower()
test_ds = Dataset.from_pandas(test_df)

dev_df = pd.read_csv('./dataset/final/dev.csv', usecols=['source', 'label'])
dev_df['source'] = dev_df['source'].str.lower()
dev_df['label'] = dev_df['label'].str.lower()
dev_ds = Dataset.from_pandas(dev_df)

all_df = pd.concat((train_df, test_df, dev_df))
all_ds = Dataset.from_pandas(all_df)

In [3]:
tokenizer = AutoTokenizer.from_pretrained('./results', local_files_only=True)
model = AutoModelForSeq2SeqLM.from_pretrained('./results', local_files_only=True)

In [ ]:
a = 0
t = 0

for source, label in zip(test_df['source'], test_df['label']):
    input_ids = tokenizer(source, return_tensors="pt", max_length=512, padding=True, truncation=True).input_ids 
    outputs = model.generate(input_ids)
    
    decoded = ''.join(tokenizer.convert_ids_to_tokens(outputs[0])[1:-1]).replace('▁', ' ').strip()
    
    t += 1

    if decoded == label:
        a += 1
    else:
        print(source)
        print(decoded)
        print(label)
        print()

apt_booking_id - int , apt_id - int , guest_id - int , booking_status_code - str , booking_start_date - str , booking_end_date - str | plot how many booking start date by grouped by booking start date as a bar graph , order y in ascending order .
mark bar encoding x booking_start_date y aggregate count booking_start_date transform sort y asc bin x by year
mark bar encoding x booking_start_date y aggregate count booking_start_date transform sort y asc bin x by weekday

staff_id - float , project_id - int , role_code - str , date_from - str , date_to - str , other_details - float | when do all the researcher role staff start to work ? bin the time into year interval and count them with a line chart , and order from low to high by the date_from please .
mark line encoding x date_to y aggregate count date_to transform sort x asc bin x by year
mark line encoding x date_from y aggregate count date_from transform sort x asc bin x by year

company_id - int , rank - int , company - str , headqu

season - float , player - str , position - str , country - int , team - int , draft_pick_number - int , draft_class - str , college - str | show the draft pick numbers and draft classes of players whose positions are defenders in a bar chart , and list by the draft_class from high to low .
mark bar encoding x position y aggregate count position transform sort x desc
mark bar encoding x draft_class y aggregate none draft_pick_number transform filter position = "defender" sort x desc

customer_event_id - int , customer_id - int , date_moved_in - str , property_id - int , resident_id - int , thing_id - int | bar chart x axis date moved in y axis how many date moved in , list in ascending by the y axis please .
mark bar encoding x date_moved_in y aggregate count date_moved_in transform sort y asc bin x by weekday
mark bar encoding x date_moved_in y aggregate count date_moved_in transform sort y asc bin x by year

id - int , series_name - str , country - str , language - str , content - str

cust_id - int , cust_name - str , acc_type - str , acc_bal - int , no_of_loans - int , credit_score - int , branch_id - int , state - str | can you compare the account balances of customers with the letter 'a' in their names using a bar graph , and order by the x axis in desc please .
mark bar encoding x acc_type y aggregate none acc_bal transform filter credit_score = "a" sort x desc
mark bar encoding x cust_name y aggregate none acc_bal transform filter cust_name like '%a%' sort x desc

registration_id - int , student_id - int , course_id - int , date_of_enrolment - str , date_of_completion - str | how many enrolment students in each month ? return a bar chart binning date of enrolment by month interval .
mark bar encoding x date_of_completion y aggregate count date_of_completion transform bin x by month
mark bar encoding x date_of_enrolment y aggregate count date_of_enrolment transform bin x by month

id - int , name - str , lat - float , long - float , dock_count - int , city - str

tourist_attraction_id - int , attraction_type_code - int , location_id - int , how_to_get_there - str , name - str , description - float , opening_hours - float , other_details - float | show different ways to get to attractions and the number of attractions that can be accessed in the corresponding way . plot them as bar chart .
mark bar encoding x location_to_get_there y aggregate count location_to_get_there transform
mark bar encoding x how_to_get_there y aggregate count how_to_get_there transform

people_id - int , sex - str , name - str , date_of_birth - str , height - float , weight - float | what is the averag weights for people of each sex ? show a bar chart , i want to rank by the y axis from low to high .
mark bar encoding x sex y aggregate none weight transform sort y asc
mark bar encoding x sex y aggregate mean weight transform sort y asc

student_id - int , date_of_registration - str , date_of_latest_logon - str , login_name - str , password - str , personal_name - str , m

staff_id - float , project_id - int , role_code - str , date_from - str , date_to - str , other_details - float | give me the trend about the date_from attributes of all researchers , and bin the date into year interval , rank by the x axis in desc .
mark line encoding x date_from y aggregate count date_to transform sort x desc bin x by year
mark line encoding x date_from y aggregate count date_from transform sort x desc bin x by year

company_id - int , rank - int , company - str , headquarters - str , main_industry - str , sales_billion - float , profits_billion - float , assets_billion - float , market_value - float | show the number of companies whose headquarters are not from usa fpr each main industry in a bar chart , and display y axis from low to high order please .
mark bar encoding x headquarters y aggregate count headquarters transform sort y asc
mark bar encoding x main_industry y aggregate count main_industry transform filter headquarters != 'usa' sort y asc

transaction_i

...

flno - int , origin - str , destination - str , distance - int , departure_date - str , arrival_date - str , price - float , aid - int | a bar chart showing the number of flights with price higher than 300 for each destination city , show from low to high by the bar .
mark bar encoding x destination y aggregate count destination transform filter destination > 300 sort x asc
mark bar encoding x destination y aggregate count destination transform filter price > 300 sort x asc

dog_id - int , owner_id - int , abandoned_yn - int , breed_code - str , size_code - str , name - str , age - int , date_of_birth - str , gender - int , weight - float , date_arrived - str , date_adopted - str , date_departed - str | what is the weight of each dog that was abandoned ? show in a pie chart .
mark arc encoding x name y aggregate none weight transform filter abandoned_yn = 1
mark arc encoding x name y aggregate none age transform filter abandoned_yn = 1

exhibition_id - int , year - int , theme - str , 

tourist_attraction_id - int , attraction_type_code - int , location_id - int , how_to_get_there - str , name - str , description - float , opening_hours - float , other_details - float | list all the possible ways to get to attractions , together with the number of attractions accessible by these methods in a bar chart , could you order y axis from low to high order ?
mark bar encoding x location_type_code y aggregate count location_type_code transform sort y asc
mark bar encoding x how_to_get_there y aggregate count how_to_get_there transform sort y asc

catalog_entry_id - int , catalog_level_number - int , parent_entry_id - int , previous_entry_id - int , next_entry_id - int , catalog_entry_name - str , product_stock_number - str , price_in_dollars - float , price_in_euros - float , price_in_pounds - float , capacity - int , length - int , height - int , width - int | visualize a bar chart for which catalog contents has price above 700 dollars ? show their catalog entry names and cap

customer_id - int , customer_address_id - int , customer_status_code - str , date_became_customer - str , date_of_birth - str , first_name - str , last_name - str , amount_outstanding - float , email_address - str , phone_number - str , cell_mobile_phone_number - str | when are the birthdays of customer who are classified as 'good customer' status , and count them by a line chart , and show from low to high by the x-axis please .
mark line encoding x date_of_birth y aggregate count date_of_birth transform filter customer_status_code = 'good customer' sort x asc
mark line encoding x date_of_birth y aggregate count date_of_birth transform filter customer_status_code = 'good customer' sort x asc bin x by year

id - int , series_name - str , country - str , language - str , content - str , pixel_aspect_ratio_par - str , hight_definition_tv - str , pay_per_view_ppv - str , package_option - str | what are the pixel aspect ratio and country of origin for all tv channels that do not use englis

mission_id - int , ship_id - int , code - str , launched_year - int , location - str , speed_knots - int , fate - str | show the number of fate from each fate , i want to show by the x-axis in descending .
mark bar encoding x lot y aggregate count lot transform sort x desc
mark bar encoding x fate y aggregate count fate transform sort x desc

apt_id - int , building_id - int , apt_type_code - str , apt_number - str , bathroom_count - int , bedroom_count - int , room_count - int | what are the apartment number and the room count of each apartment visualize by bar chart , and list x-axis from high to low order .
mark bar encoding x apt_type_code y aggregate count apt_type_code transform sort x desc
mark bar encoding x apt_number y aggregate none room_count transform sort x desc

document_type_code - str , document_type_name - str , document_type_description - str | bar graph to show the number of document type description from different document type description , and show in asc by the 

team_id - int , school_id - int , team_name - str , acc_regular_season - str , acc_percent - float , acc_home - str , acc_road - str , all_games - str , all_games_percent - float , all_home - str , all_road - str , all_neutral - str | school_id across all road attribute in a pie chart .
mark arc encoding x school_id y aggregate none school_id
mark arc encoding x all_road y aggregate none school_id

catalog_entry_id - int , catalog_level_number - int , parent_entry_id - int , previous_entry_id - int , next_entry_id - int , catalog_entry_name - str , product_stock_number - str , price_in_dollars - float , price_in_euros - float , price_in_pounds - float , capacity - int , length - int , height - int , width - int | find the name and capacity of products with price greater than 700 (in usd) show bar chart , and show y-axis in ascending order .
mark bar encoding x name y aggregate none capacity transform filter price_in_dollars > 700 sort y asc
mark bar encoding x catalog_entry_name y aggr

In [ ]:
a / t